In [ ]:
%pip install requests PyPDF2

In [ ]:
import requests
import pandas as pd
import PyPDF2
import io
import re
import base64
from IPython.display import HTML

In [ ]:
SSL_VERIFY = True
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
SSL_VERIFY = False

if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()

In [ ]:
# download CSV
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(None, index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<h3><a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a><h3>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
# query Geschäft with search term
query = {'suchBegriff': 'Geschäftsbericht des Stadtrats 2011'}
headers = {'Accept': 'application/json'}
r = requests.get('https://www.gemeinderat-zuerich.ch/api/Geschaeft', params=query, headers=headers, verify=SSL_VERIFY)
result = r.json()
result

In [ ]:
# get Geschäft details with gid
query = {'gid': result['Geschaefte'][0]['GeschaeftId']}
headers = {'Accept': 'application/json'}
r = requests.get('https://www.gemeinderat-zuerich.ch/api/Geschaeft/details', params=query, headers=headers, verify=SSL_VERIFY)
result = r.json()
result

In [ ]:
# get PDF of vote
docs = []
for step in result['Ablaufschritte']:
    docs.extend(step['AblaufschrittDokumente'])
docs
vote_doc = [doc for doc in docs if 'Schlussabstimmung' in doc['Dokumenttitel']][0]
vote_doc

In [ ]:
#https://www.gemeinderat-zuerich.ch/Geschaefte/detailansicht-geschaeft/Dokument/19d3cc38-65de-4a6a-84eb-aeb7808e4c3d/2012_0172%20Schlussabstimmung.pdf
pdf_url = f"https://www.gemeinderat-zuerich.ch/Geschaefte/detailansicht-geschaeft/Dokument/{vote_doc['Id']}/{vote_doc['Dokumenttitel']}"
r = requests.get(pdf_url, verify=SSL_VERIFY)

texts = []
with io.BytesIO(r.content) as f:
    reader = PyPDF2.PdfFileReader(f)
    
    num_pages = reader.numPages
    count = 0
    while count < num_pages:
        page = reader.getPage(count)
        count = count + 1
        text = page.extractText()
        texts.append(text)


In [ ]:
texts[0]

In [ ]:
lines = []
for text in texts:
    matches = re.findall(r'(\d{3})\s+(\w+)\s+(.+?)\s+(\w+)\s+(JA|NEIN|ENTHALTEN|--)', text)
    lines.extend(matches)

records = [{'platz': l[0], 'vorname': l[2], 'name': l[1], 'partei': l[3], 'stimme': l[4] if l[4] != '--' else ''} for l in lines]
records

In [109]:
votes = pd.DataFrame(records)
votes = votes.reindex(columns=['platz','name','vorname','partei','stimme'])
votes

,platz,name,vorname,partei,stimme
0,002,Abele,Martin,Grüne,JA
1,032,Altinay,Petek,SP,JA
2,151,Amacker,Bruno,SVP,NEIN
3,051,Ammann,Jürg,Grüne,JA
4,084,Angst,Walter,AL,ENTHALTEN
5,110,Anhorn,Ruth,SVP,NEIN
6,013,Aubert,Marianne,SP,JA
7,060,Bär,Linda,SP,JA
8,137,Bartholdi,Roger,SVP,NEIN
9,168,Baumer,Michael,FDP,JA


In [110]:
create_download_link(votes)